In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, roc_auc_score, make_scorer

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from tqdm import tqdm
from functools import reduce
from copy import deepcopy
import gc
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 

# %cd ../

In [103]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/test_ids.csv       
  inflating: data/y_prepared.csv     
  inflating: data/X_prepared.csv     
  inflating: data/test_df_prepared.csv  


In [45]:
def delete_double_spaces(nm):
    new_nm = ''
    for char in nm:
        if char != ' ' or (len(new_nm) and new_nm[-1] != ' '):
            new_nm += char
    return new_nm.strip()


def label_process(nm):
    nm = nm.lower()
    new_nm = ''
    balance = 0
    for char in nm:
        if char in '({[':
            balance += 1
        elif char in ')}]':
            balance = max(0, balance - 1)
        elif balance == 0:
            new_nm += char
    nm = new_nm
    nm = delete_double_spaces(nm)
    return nm

In [46]:
df = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
test_ids = deepcopy(test_data.id)

FileNotFoundError: [Errno 2] File b'data/train.csv' does not exist: b'data/train.csv'

In [ ]:
df['processed_name'] = df.name_mfi.apply(label_process)
test_data['processed_name'] = test_data.name_mfi.apply(label_process)

In [ ]:
df['item_hash'] = df['name_mfi'].apply(hash) ^ df.weight_mfi.apply(lambda x: hash(str(x))) ^ df.weight.apply(lambda x: hash(str(x))) ^ \
    df.mailtype.apply(lambda x: hash(str(x))) ^ df.transport_pay.apply(lambda x: hash(str(x)))

test_data['item_hash'] = test_data['item_hash'] = test_data['name_mfi'].apply(hash) ^ test_data.weight_mfi.apply(lambda x: hash(str(x))) ^ test_data.weight.apply(lambda x: hash(str(x))) ^ \
    test_data.mailtype.apply(lambda x: hash(str(x))) ^ test_data.transport_pay.apply(lambda x: hash(str(x)))

In [ ]:
hash_counter = Counter(pd.concat([df.item_hash, test_data.item_hash], ignore_index=True))
name_counter = Counter(pd.concat([df.processed_name, test_data.processed_name], ignore_index=True))

In [ ]:
id_hash = pd.concat([df[['id', 'item_hash']], test_data[['id', 'item_hash']]], ignore_index=True)
id_hash = id_hash.groupby('item_hash').agg(max).rename(columns={'id': 'last_id'})

In [ ]:
df = df.join(id_hash, on='item_hash')
test_data = test_data.join(id_hash, on='item_hash')

In [ ]:
X = df.drop(columns=["label"])
y = df.label
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
top_operators = [obj for obj, count in Counter(test_data['index_oper'].replace(' ', '0').astype(float).astype(int)).most_common()][:10]

In [ ]:
df.item_hash.nunique()

In [ ]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

def prepare_data(df):
    oper_type, oper_attr = zip(*df['oper_type + oper_attr'].map(lambda x: x.split('_')))
    
    # df['is_negative_oper_attr'] = pd.Series(oper_attr) == '-1'
    df['specialization_ratio'] = df.total_qty_over_index_and_type / df.total_qty_over_index.clip(lower=1)
    df['total_over_person'] = df.total_qty_over_index / df.dist_qty_oper_login_1.clip(lower=1)
    df['unknown_oper_ratio'] = df.total_qty_oper_login_0 / df.total_qty_oper_login_1.clip(lower=1)
    
    df['same_hash'] = df.item_hash.apply(lambda h: hash_counter[h])
    df['same_name'] = df.processed_name.apply(lambda name: name_counter[name])
    
    df['isnt_last'] = (df.id == df.last_id) * (df.same_hash < 30)
    
    for operator in top_operators:
        df[f'is_{operator}_operator'] = df['index_oper'].replace(' ', '0').astype(float).astype(int) == operator
    
    replace_zeros = ['priority', 'is_privatecategory', 'is_in_yandex', 'mailtype', 'directctg']
    for column in replace_zeros:
        df[column][df[column] == 0] = df[column].mode()[0]
        df[column][df[column] == '0'] = df[column].mode()[0]
    
    df.priority = 1 * (df.priority == 7503.) + 2 * (df.priority == 7504.) + 3 * (df.priority == 7506.)
    
    df.index_oper = df.index_oper.replace(' ', '0').astype(float).astype(int)
    
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
    df['name_mfi_count'] = df.name_mfi.apply(lambda name: 0 if name == '0' else len(name.split(',')))
    
    df.is_return = df.is_return.apply(lambda text: text == 'Y')
        
    df.drop(columns=["oper_type + oper_attr"], inplace=True)
    df.drop(columns=["name_mfi", "item_hash", "processed_name"], inplace=True)
    df.drop(columns=['mailrank'], inplace=True)
    df.drop(columns=useless, inplace=True, errors='ignore')
    
    return df

In [ ]:
X_train = prepare_data(X_train)
X_val = prepare_data(X_val)
test_df = prepare_data(test_data)

In [ ]:
X_train.to_csv('data/X_train_prepared.csv', index=False)
X_val.to_csv('data/X_val_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)
test_df.to_csv('data/test_df_prepared.csv', index=False)
y_train.to_csv('data/y_train_prepared.csv', index=False)
y_val.to_csv('data/y_val_prepared.csv', index=False)

In [112]:
useless = []
cat_features = ["type", "is_privatecategory", "class", "is_in_yandex", "mailctg", "directctg", "mailtype"]
cat_features = list(set(cat_features) - set(useless))

cat_features = list(set(cat_features) - set(useless))
def to_cat_features(df):
    for cat_feature in cat_features:
        df[cat_feature] = df[cat_feature].apply(lambda text: str(text))
        
# X_train = pd.read_csv('data/X_train_prepared.csv')
# y_train = pd.read_csv('data/y_train_prepared.csv')
# X_val = pd.read_csv('data/X_val_prepared.csv')
# y_val = pd.read_csv('data/y_val_prepared.csv')
# test_df = pd.read_csv('data/test_df_prepared.csv')
# test_ids = pd.read_csv('data/test_ids.csv')

X_train = pd.read_csv('data/X_prepared.csv') 
y_train = pd.read_csv('data/y_prepared.csv') 
test_df = pd.read_csv('data/test_df_prepared.csv')
test_ids = pd.read_csv('data/test_ids.csv')


to_cat_features(X_train)
# to_cat_features(X_val)
to_cat_features(test_df)

In [113]:
model = CatBoostClassifier(random_state=0, max_depth=7, verbose=200, iterations=15000, auto_class_weights="Balanced", cat_features=cat_features)

In [114]:
#!c1.32
model.fit(X_train, y_train)

Learning rate set to 0.035296
0:	learn: 0.6502582	total: 1.09s	remaining: 4h 33m 9s
200:	learn: 0.2346547	total: 2m 54s	remaining: 3h 34m 37s
400:	learn: 0.2276985	total: 5m 48s	remaining: 3h 31m 16s
600:	learn: 0.2236196	total: 8m 48s	remaining: 3h 30m 59s
800:	learn: 0.2209645	total: 11m 50s	remaining: 3h 29m 54s
1000:	learn: 0.2188890	total: 14m 53s	remaining: 3h 28m 20s
1200:	learn: 0.2171643	total: 17m 58s	remaining: 3h 26m 36s
1400:	learn: 0.2156608	total: 21m 5s	remaining: 3h 24m 39s
1600:	learn: 0.2144128	total: 24m 13s	remaining: 3h 22m 48s
1800:	learn: 0.2131980	total: 27m 22s	remaining: 3h 20m 37s
2000:	learn: 0.2120733	total: 30m 30s	remaining: 3h 18m 10s
2200:	learn: 0.2110033	total: 33m 38s	remaining: 3h 15m 36s
2400:	learn: 0.2099614	total: 36m 48s	remaining: 3h 13m 10s
2600:	learn: 0.2089022	total: 39m 54s	remaining: 3h 10m 16s
2800:	learn: 0.2079234	total: 43m	remaining: 3h 7m 17s
3000:	learn: 0.2070159	total: 46m 2s	remaining: 3h 4m 6s
3200:	learn: 0.2061374	total: 49

In [115]:
def score_model(model, X, y, threshold=0.5):  # 0.225 is the best for now
    pred = model.predict_proba(X)[:, 1] > threshold
    recall = recall_score(y, pred, average="macro")
    roc_auc = roc_auc_score(y,  pred, multi_class='ovo')
    score = 0.1 * recall + 0.9 * roc_auc
    return score

In [116]:
score_model(model, X_train, y_train, 0.35)

0.9242626158854033

In [117]:
sorted(list(zip(X_train.columns, model.feature_importances_)), key = lambda x: -x[1])

[('index_oper', 7.9108946256050325),
 ('specialization_ratio', 7.3056499193324145),
 ('total_over_person', 7.062019861244038),
 ('unknown_oper_ratio', 7.013490696292854),
 ('same_oper_item_hash', 6.940572921718261),
 ('total_qty_oper_login_0', 6.620282085867345),
 ('total_qty_oper_login_1', 5.590515073847468),
 ('mailctg', 5.236052254313247),
 ('same_characteristics_hash', 5.171715851408008),
 ('same_hash', 4.217497688931641),
 ('transport_pay', 4.195507968136628),
 ('type', 4.021732578254022),
 ('total_qty_over_index_and_type', 3.0964834127344916),
 ('weight', 2.86816507298373),
 ('dist_qty_oper_login_1', 2.603680082277148),
 ('total_qty_over_index', 2.5550175234687535),
 ('weight_mfi', 2.3652648850098608),
 ('price_mfi', 1.8169604870988738),
 ('same_name', 1.7627978470730432),
 ('id', 1.4259322999914417),
 ('name_mfi_len', 1.4181978636529873),
 ('last_id', 1.4014551230017014),
 ('till_end', 1.2755155523166402),
 ('priority', 1.1137063750239997),
 ('name_mfi_count', 0.9796175063712711

In [118]:
test_preds = (model.predict_proba(test_df)[:, 1])
submission = pd.DataFrame({'id': test_ids.id, 'label': test_preds})
submission.to_csv("submission2.csv", index=False)

In [119]:
7

7

In [ ]:
unrounded_sub = pd.read_csv('submission.csv') 
sub = unrounded_sub.copy()
sub.label = (unrounded_sub.label > 0.225).astype(int) 
sub.to_csv('submission_rounded.csv', index=False)

In [ ]:
sub.label.mean()
unrounded_sub.label.mean()